In [28]:

from laserembeddings import Laser
from os import path
import pandas as pd
from tqdm import tqdm_notebook,tqdm
#import demoji
#demoji.download_codes()
%load_ext autoreload
%autoreload 2
import numpy as np

In [92]:
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='macro'))
    avg.append(accuracy_score(y_true, y_pred, normalize=True))
    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support','accuracy']
    list_all=list(metrics_summary)
    list_all.append(cm.diagonal())
    class_report_df = pd.DataFrame(
        list_all,
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-2] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T

In [104]:
parent_path='../Data/New_Data_15-06-2020/'

In [105]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')
# annotated_df=remove_duplicates_within(annotated_df)
print(len(annotated_df))

4782


In [ ]:
list_text = []
lang_text = []
for index,row in tqdm(annotated_df.iterrows(),total=len(annotated_df)):
    if len(row['message_text'])>4000:
        string=row['message_text'][0:2000]+row['message_text'][-2000:]
        list_text.append(demoji.replace(string,repl=""))
    else:
        list_text.append(demoji.replace(row["message_text"],repl=""))
    lang_text.append(row["language"])

### LASER

In [ ]:
laser = Laser()

embed_list=[]

length_given=len(list_text)
batch_size=64
for i in tqdm(range(0,length_given,batch_size)):
    if(i+batch_size<=length_given):
        temp_text=list_text[i:i+batch_size]
        temp_lang=lang_text[i:i+batch_size]
    else:
        temp_text=list_text[i:length_given]
        temp_lang=lang_text[i:length_given]
    embeddings = laser.embed_sentences(temp_text,lang=temp_lang)  # lang is only used for tokenization
    embed_list+=list(embeddings)

In [31]:
X_0 = np.array(embed_list)
y_0 = np.array(annotated_df['one_fear_speech'])

NameError: name 'embed_list' is not defined

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix,make_scorer, f1_score, accuracy_score, recall_score, precision_score, roc_auc_score,classification_report, precision_recall_fscore_support

from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import random

In [26]:
import emoji
from tqdm import tqdm_notebook
from gensim.models.phrases import Phrases, Phraser


# In[6]:


#export
def extract_emojis(str1):
    try:
        return [c for c in str1 if c in emoji.UNICODE_EMOJI]
    except AttributeError:
        return []


In [12]:
acc=[]
macro_f1=[]
prec=[]
recall=[]
prob=[]

skf = StratifiedKFold(n_splits=10, random_state= 2020)

for train_index, test_index in skf.split(X_0, y_0):
    print("TRAIN:", train_index[0:5], "TEST:", test_index[0:5])
    X_train, X_test = X_0[train_index], X_0[test_index]
    y_train, y_test = y_0[train_index], y_0[test_index]
    class_weights = dict(zip(np.unique(y_train), compute_class_weight("balanced", np.unique(y_train),y_train)))
    print(class_weights)
    classifier= LogisticRegression(class_weight=class_weights)
    #classifier=SVC(class_weight=class_weights,kernel='rbf',probability=True)
    classifier.fit(X_train, y_train)
    y_pred=classifier.predict(X_test)
    acc.append(accuracy_score(y_test, y_pred))
    macro_f1.append(f1_score(y_test, y_pred, average='macro'))
    prec.append(precision_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    prob.append(classifier.predict_proba(X_test))


TRAIN: [446 447 448 449 450] TEST: [0 1 2 3 4]
{0: 0.6567460317460317, 1: 2.0949367088607596}


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


ValueError: setting an array element with a sequence.

### For SVC (with rbf kernel)

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

### For logisitic regression

In [30]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))

Accuracy: 0.72 (+/- 0.09)
Macro F1: 0.66 (+/- 0.08)
Precision for +ve class: 0.45 (+/- 0.16)
Recall for +ve class: 0.59 (+/- 0.13)


### Doc2Vec


In [108]:
from utils.preprocess import *
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

list_sents=[preprocess_doc(ele,params={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': True}) for ele in tqdm(annotated_df['message_text'],total=len(annotated_df))]
#list_emojis = [" ".join(extract_emojis(ele)) for ele in tqdm(annotated_df['message_text'],total=len(annotated_df))]

100%|██████████| 4782/4782 [01:03<00:00, 75.11it/s] 


In [114]:
list_sents=[1,2,3,4]
list_sents[-2:]

[3, 4]

In [49]:
X_0 = np.array(list_sents,dtype='object')
X_1 = np.array(list_emojis,dtype='object')
y_0 = np.array(annotated_df['one_fear_speech'])

In [53]:
X_3= np.vstack((X_0, X_1))

In [59]:
X_3=np.transpose(X_3)

In [102]:
acc=[]
macro_f1=[]
prec=[]
recall=[]
prob=[]
auc_roc=[]
list_total_preds=[]
list_total_truth=[]
skf = StratifiedKFold(n_splits=5, random_state= 2020)

for train_index, test_index in skf.split(X_3, y_0):
    print("TRAIN:", train_index[0:5], "TEST:", test_index[0:5])
    X_train, X_test = X_3[train_index], X_3[test_index]
    y_train, y_test = y_0[train_index], y_0[test_index]
    
    
    class_weights = dict(zip(np.unique(y_train), compute_class_weight("balanced", np.unique(y_train),y_train)))
    
    
    print(class_weights)
    ### Generate doc2vec vectors
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(X_train[:,0])]
    model = Doc2Vec(documents, vector_size=300, window=5, min_count=1, workers=10)
    X_train_embed = np.array([list(model.infer_vector(ele)) for ele in X_train[:,0]])
    X_test_embed = np.array([list(model.infer_vector(ele)) for ele in X_test[:,0]])
    
    ##generate emoji vectors
    tf_idf_vec_smooth = TfidfVectorizer(use_idf=True,  
                        smooth_idf=True,  
                        ngram_range=(1,1),min_df=0.01,max_df=0.95,analyzer='char')

    X_train_tfidf_emoji = tf_idf_vec_smooth.fit_transform(X_train[:,1]).toarray()
    X_test_tfidf_emoji = tf_idf_vec_smooth.transform(X_test[:,1]).toarray()
    
    print(X_train_tfidf_emoji.shape,X_train_embed.shape)
    
    X_train_embed_2= np.concatenate((X_train_embed, X_train_tfidf_emoji),axis=1)
    X_test_embed_2= np.concatenate((X_test_embed, X_test_tfidf_emoji),axis=1)
    
    
    classifier= LogisticRegression(class_weight='balanced',max_iter=500)
    #classifier=SVC(class_weight='balanced',kernel='rbf',probability=True)
    classifier.fit(X_train_embed_2, y_train)
    y_pred=classifier.predict(X_test_embed_2)
    y_pred_proba = classifier.predict_proba(X_test_embed_2)
    acc.append(accuracy_score(y_test, y_pred))
    macro_f1.append(f1_score(y_test, y_pred, average='macro'))
    auc_roc.append(roc_auc_score(y_test, y_pred_proba[:,1],average='macro'))
    prec.append(precision_score(y_test, y_pred))
    recall.append(recall_score(y_test, y_pred))
    prob.append(classifier.predict_proba(X_test_embed_2))
    list_total_preds+=list(y_pred)
    list_total_truth+=list(y_test)


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


TRAIN: [870 872 873 874 875] TEST: [0 1 2 3 4]
{0: 0.6567651098901099, 1: 2.0947426067907995}
(3825, 63) (3825, 300)
TRAIN: [0 1 2 3 4] TEST: [870 872 873 874 875]
{0: 0.6567651098901099, 1: 2.0947426067907995}
(3825, 54) (3825, 300)
TRAIN: [0 1 2 3 4] TEST: [1771 1772 1773 1774 1775]
{0: 0.6569368131868132, 1: 2.0929978118161925}
(3826, 60) (3826, 300)
TRAIN: [0 1 2 3 4] TEST: [2757 2758 2759 2761 2762]
{0: 0.6569368131868132, 1: 2.0929978118161925}
(3826, 60) (3826, 300)
TRAIN: [0 1 2 3 4] TEST: [3792 3794 3796 3799 3801]
{0: 0.6569368131868132, 1: 2.0929978118161925}
(3826, 67) (3826, 300)


### SVC


In [101]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Auc Roc F1: %0.2f (+/- %0.2f)" % (np.mean(auc_roc), np.std(auc_roc) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))
print(pandas_classification_report(list_total_truth, list_total_preds))


Accuracy: 0.76 (+/- 0.05)
Macro F1: 0.69 (+/- 0.04)
Auc Roc F1: 0.78 (+/- 0.06)
Precision for +ve class: 0.50 (+/- 0.12)
Recall for +ve class: 0.62 (+/- 0.13)
             precision    recall  f1-score  support  accuracy
0             0.869928  0.801099  0.834096   3640.0  0.801099
1             0.493706  0.618214  0.548989   1142.0  0.618214
avg / total   0.681817  0.709656  0.691543   4782.0  0.757424


### Logistic Regression

In [103]:
print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(acc), np.std(acc) * 2))
print("Macro F1: %0.2f (+/- %0.2f)" % (np.mean(macro_f1), np.std(macro_f1) * 2))
print("Auc Roc: %0.2f (+/- %0.2f)" % (np.mean(auc_roc), np.std(auc_roc) * 2))
print("Precision for +ve class: %0.2f (+/- %0.2f)" % (np.mean(prec), np.std(prec) * 2))
print("Recall for +ve class: %0.2f (+/- %0.2f)" % (np.mean(recall), np.std(recall) * 2))
print(pandas_classification_report(list_total_truth, list_total_preds))


Accuracy: 0.72 (+/- 0.08)
Macro F1: 0.66 (+/- 0.06)
Auc Roc: 0.76 (+/- 0.06)
Precision for +ve class: 0.45 (+/- 0.13)
Recall for +ve class: 0.62 (+/- 0.11)
             precision    recall  f1-score  support  accuracy
0             0.862975  0.749176  0.802059   3640.0  0.749176
1             0.437115  0.620841  0.513025   1142.0  0.620841
avg / total   0.650045  0.685008  0.657542   4782.0  0.718528


In [ ]:
documents

### Emoji extraction


In [7]:
import pandas as pd
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')
# annotated_df=remove_duplicates_within(annotated_df)
print(len(annotated_df))

4782


In [9]:
emoji_sents=[]
for index,row in annotated_df.iterrows():
    str1=" ".join(extract_emojis(row['message_text']))
    emoji_sents.append(str1)

With Smoothing:
                       🇮         🇳    🏻    👇         🙏         🚩
0     0.962035  0.050357  0.050217  0.0  0.0  0.045789  0.259488
1     0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000
2     0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000
3     0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000
4     0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000
...        ...       ...       ...  ...  ...       ...       ...
4777  0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000
4778  0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000
4779  0.592966  0.362112  0.361106  0.0  0.0  0.000000  0.621988
4780  0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000
4781  0.000000  0.000000  0.000000  0.0  0.0  0.000000  0.000000

[4782 rows x 7 columns]
